In [1]:
import pymysql
import pandas as pd
import numpy as np

In [2]:
def get_df_from_sql(sql):
    # Connect to the MySQL database
    db = pymysql.connect(host='localhost',
                         port=3306,
                         user='root',
                         passwd='root',
                         db='originaldata')
    df = pd.read_sql(sql = sql, con = db)
    return df

In [ ]:
sql_alias = """SELECT * FROM alias"""
data_alias = get_df_from_sql(sql_alias)
sql_PL = """SELECT * FROM Patientlocation"""
data_PL = get_df_from_sql(sql_PL)
# data_alias

In [65]:
data_PL

,PatientLocationSerNum,AppointmentSerNum,AliasSerNum,PatientLocationAriaSer,ResourceSer,RevCount,CheckedInFlag,ArrivalDateTime,LastUpdated
0,1,334205,2,390228,8225,0,1,2015-06-15 09:10:00,2015-08-27 00:00:07
1,2,334232,2,396112,8225,0,1,2015-07-23 09:24:00,2015-08-27 00:00:07
2,3,330331,2,354131,6280,8,1,2014-10-22 09:02:00,2015-08-27 00:00:07
3,4,334698,2,371951,6280,25,1,2015-02-18 11:18:00,2015-08-27 00:00:07
4,5,330438,2,359916,5485,9,1,2014-11-25 13:35:00,2015-08-27 00:00:07
5,6,330403,2,357567,6279,12,1,2014-11-11 13:21:00,2015-08-27 00:00:07
6,7,330929,2,359168,6277,9,1,2014-11-20 11:57:00,2015-08-27 00:00:07
7,8,334702,2,374660,6277,22,1,2015-03-05 12:11:00,2015-08-27 00:00:07
8,9,332583,2,360363,6277,9,1,2014-11-27 11:43:00,2015-08-27 00:00:07
9,10,332544,2,362540,128,12,1,2014-12-11 10:39:00,2015-08-27 00:00:07


In [4]:
data_PL_grouped = data_PL.groupby('AppointmentSerNum')
data_PL_grouped.count()['PatientLocationSerNum'].max()

1

In [15]:
PL_alias = pd.DataFrame({'AliasSerNum': list(data_PL.AliasSerNum.value_counts().index),
                         'appointment_count': data_PL.AliasSerNum.value_counts().tolist()})
PL_alias = pd.merge(PL_alias, data_alias[['AliasSerNum', 'AliasName', 'AliasType']],
                    on = 'AliasSerNum', how = 'inner')
PL_alias

,AliasSerNum,appointment_count,AliasName,AliasType
0,23,136914,Treatment Appointment,Appointment
1,40,26789,Follow-up Appointments,Appointment
2,31,13365,First Treatment,Appointment
3,2,12060,Consult Appointment,Appointment
4,3,11636,Ct-Sim,Appointment
5,48,6047,Machine Warmup Morning Appt,Appointment
6,41,44,Intra-Treatment Appointment,Appointment


##### 说明一个AppointmentSerNum 只对应一个PatientLocationSerNum，按照之前统计的pltArriveDateTime 应该是check-in 时刻
##### 一个AppointmentSerNum 只对应一个pltArriveDateTime，pltArriveDateTime 在ActualStartDate 之前）

In [5]:
sql_PLMH = """SELECT * FROM Patientlocationmh"""
data_PLMH = get_df_from_sql(sql_PLMH)

In [6]:
data_PLMH_grouped = data_PLMH.groupby('AppointmentSerNum')
data_PLMH_grouped.count()['PatientLocationMHSerNum'].mean()

11.257830406732388

##### 说明一个AppointmentSerNum 只对应多个PatientLocationMHSerNum，平均为11.257830406732388

In [16]:
PLMH_alias = pd.DataFrame({'AliasSerNum': list(data_PLMH.AliasSerNum.value_counts().index),
                           'PLMH_count': data_PLMH.AliasSerNum.value_counts().tolist()})
PLMH_alias = pd.merge(PLMH_alias, data_alias[['AliasSerNum', 'AliasName', 'AliasType']],
                      on = 'AliasSerNum', how = 'inner')
PLMH_alias

,AliasSerNum,PLMH_count,AliasName,AliasType
0,23,1589263,Treatment Appointment,Appointment
1,40,284647,Follow-up Appointments,Appointment
2,31,172095,First Treatment,Appointment
3,2,143519,Consult Appointment,Appointment
4,3,84181,Ct-Sim,Appointment
5,48,58996,Machine Warmup Morning Appt,Appointment
6,41,338,Intra-Treatment Appointment,Appointment


##### PatientLocationMHSerNum 里Alias 的类型和数量

In [9]:
# 看一下时间点集合里，有多少个不一样的时间点
def count_ADT(df, time_type = 'ArrivalDateTime'):
    len_ADT = len(set(df[time_type].tolist()))
    return len_ADT

In [10]:
for num in PLMH_alias.AliasSerNum.tolist():
    print(f'\n{num}')
    data_PLMH_num = data_PLMH[data_PLMH.AliasSerNum == num]
    data_PLMH_num_grouped = data_PLMH_num.groupby('AppointmentSerNum')
    # data_PLMH_23_grouped.get_group(227302)

    ADT_num = []
    for ASN in list(data_PLMH_num_grouped.groups.keys()):
        df = data_PLMH_num_grouped.get_group(ASN)
        same = count_ADT(df)
        ADT_num.append(same)

    print(pd.value_counts(ADT_num))


23
2     95720
1     30447
3      6373
4      2340
5       962
6       589
7       268
8       154
9        74
10       57
11       27
12       22
13       20
14       14
15       11
18        4
19        3
24        3
17        2
20        2
21        2
30        1
16        1
27        1
28        1
35        1
dtype: int64

40
1     9018
2     9004
3     6829
4     1331
5      461
6      206
7      116
8       58
9       30
10      22
11      17
12      13
13       6
16       3
14       2
20       1
21       1
27       1
15       1
dtype: int64

31
2     8862
1     2746
3      928
4      386
5      205
6      116
7       58
8       32
9       12
11       7
10       6
14       5
12       2
13       1
dtype: int64

2
2     6103
3     2661
1     2436
4      555
5      167
6       69
7       37
8       21
9       16
12       6
10       6
13       5
11       4
17       3
18       2
14       2
52       1
24       1
16       1
15       1
dtype: int64

3
1     9526
2     1326
3      487
4 

In [11]:
for num in PLMH_alias.AliasSerNum.tolist():
    print(f'\n{num}')
    data_PLMH_num = data_PLMH[data_PLMH.AliasSerNum == num]
    data_PLMH_num_grouped = data_PLMH_num.groupby('AppointmentSerNum')
    # data_PLMH_23_grouped.get_group(227302)

    HDT_num = []
    for ASN in list(data_PLMH_num_grouped.groups.keys()):
        df = data_PLMH_num_grouped.get_group(ASN)
        same = count_ADT(df, 'HstryDateTime')
        HDT_num.append(same)

    print(pd.value_counts(HDT_num))


23
4     78479
3     29884
5     12256
6      6555
2      4302
7      2227
8      1300
9       662
1       441
10      416
11      207
12      135
13       68
14       55
15       37
16       23
17       15
18        9
19        9
20        4
21        3
22        2
24        2
26        2
30        1
25        1
27        1
28        1
29        1
35        1
dtype: int64

40
3     8434
2     7715
4     7009
5     2162
6      711
1      480
7      273
8      161
9       65
10      36
11      24
12      16
13      13
14      10
15       2
17       2
16       2
18       2
21       1
30       1
22       1
dtype: int64

31
4     5861
3     2773
5     1776
6     1112
2      659
7      461
8      299
9      166
10     102
11      47
12      30
1       22
13      17
14      12
15      11
17       6
16       4
19       3
21       2
26       1
18       1
22       1
dtype: int64

2
3     5109
4     3081
2     1877
5     1180
6      418
7      157
1      116
8       65
9       36
10      16
11 

##### 说明并没有哪一种Alias 中，一个AppointmentSerNum 里面全是一样的ArrivalDateTime，HstryDateTime
##### 也就是，一个AppointmentSerNum 中可能有多个ArrivalDateTime，HstryDateTime

In [28]:
def count_Alias(df):
    Alias_set = set(df['AliasSerNum'].tolist())
    len_Alias = len(Alias_set)
    return len_Alias, Alias_set

In [29]:
APT_Alias_num = []
APT_Alias_2 = []
data_PLMH_grouped = data_PLMH.groupby('AppointmentSerNum')
for APT in list(data_PLMH_grouped.groups.keys()):
    df = data_PLMH_grouped.get_group(APT)
    same, Alias_set = count_Alias(df)
    APT_Alias_num.append(same)
    if same == 2:
        APT_Alias_2.append(Alias_set)
print(pd.value_counts(APT_Alias_num))

1    207061
2       176
dtype: int64


In [63]:
df_APT_Alias_2 = pd.DataFrame(APT_Alias_2).drop_duplicates()
df_APT_Alias_2.columns = ['first', 'second']
df_APT_Alias_2

,first,second
0,40,2
6,31,23
172,40,41


In [64]:
df_APT_Alias_2 = pd.merge(df_APT_Alias_2, data_alias[['AliasSerNum', 'AliasName', 'AliasType']],
                          left_on = 'first', right_on = 'AliasSerNum', how = 'inner')[['first', 'second', 'AliasName']]
df_APT_Alias_2 = pd.merge(df_APT_Alias_2, data_alias[['AliasSerNum', 'AliasName']],
                          left_on = 'second', right_on = 'AliasSerNum', how = 'inner',
                          suffixes = ('_1', '_2')).drop('AliasSerNum', 1)
df_APT_Alias_2

,first,second,AliasName_1,AliasName_2
0,40,2,Follow-up Appointments,Consult Appointment
1,40,41,Follow-up Appointments,Intra-Treatment Appointment
2,31,23,First Treatment,Treatment Appointment


##### 大部分一个AppointmentSerNum 里面的PatientLocationMHSerNum 是一种Alias
##### 少部分一个AppointmentSerNum 里面的PatientLocationMHSerNum 有两种Alias

In [26]:
data_PLMH_grouped = data_PLMH.groupby('AppointmentSerNum')
data_PLMH_grouped.get_group(866070)

,PatientLocationMHSerNum,AppointmentSerNum,AliasSerNum,PatientLocationAriaSer,ResourceSer,RevCount,CheckedInFlag,ArrivalDateTime,HstryDateTime,LastUpdated
326194,326195,866070,40,414891,8225,0,1,2015-11-12 14:16:00,2015-11-12 14:16:00,2015-11-14 01:00:55
326195,326196,866070,40,414891,8225,1,1,2015-11-12 14:16:00,2015-11-13 07:51:00,2015-11-14 01:00:55
326196,326197,866070,40,414891,8225,2,1,2015-11-12 14:16:00,2015-11-13 07:51:00,2015-11-14 01:00:55
326197,326198,866070,40,414891,8225,3,1,2015-11-12 14:16:00,2015-11-13 07:51:00,2015-11-14 01:00:55
326198,326199,866070,40,414891,8225,4,1,2015-11-12 14:16:00,2015-11-13 07:51:00,2015-11-14 01:00:55
331170,331171,866070,2,414891,8225,5,1,2015-11-12 14:16:00,2015-11-18 14:31:00,2015-11-26 05:18:45
331171,331172,866070,2,414891,8225,6,1,2015-11-12 14:16:00,2015-11-18 14:31:00,2015-11-26 05:18:45
331172,331173,866070,2,414891,8225,7,1,2015-11-12 14:16:00,2015-11-18 14:31:00,2015-11-26 05:18:45
331173,331174,866070,2,414891,8225,8,1,2015-11-12 14:16:00,2015-11-18 14:31:00,2015-11-26 05:18:45
331174,331175,866070,2,414891,8225,9,1,2015-11-12 14:16:00,2015-11-18 14:31:00,2015-11-26 05:18:45
